In [27]:
import requests
import json

In [139]:
class server_api:
    def __init__(self, user='', pwd=''):
        self.url = 'https://dolphin.jump-technology.com:3389/api/v1/'
        self.s = requests.Session()
        self.s.auth = (user, pwd)
        self.s.headers.update({'x-test': 'true'})
    def get(self, req, params):
        if type(params) is tuple:
            request = self.url + (req.format(*params))
        else:
            request = self.url + (req.format(params))
        
        self.res = ''
        try:
            self.data = self.s.get(request, headers={'x-test2': 'true'}, verify=False)
            self.res = json.loads(self.data.text)
        except:
            print('Error while connecting')
        return self.res
    def get_cached():
        return self.res

In [140]:
server = server_api('epita_user_7', 'dolphin19423')

In [141]:
server.get('asset/{}/quote?start_date={}&end_date={}', (59, '2012-01-01', '2017-06-01'))

C:\Users\cpcdoy\Anaconda3\lib\site-packages\urllib3\connectionpool.py:858: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


[{'ask': 60.02,
  'bid': 60.01,
  'close': 60.04,
  'date': '2012-01-03',
  'feed_source': 3.0,
  'gross': 60.04,
  'high': 61.87,
  'low': 59.96,
  'nav': 60.04,
  'open': 61.81,
  'pl': -1.0,
  'return': -0.0163826998689384,
  'volume': 846637.0},
 {'ask': 61.63,
  'bid': 61.62,
  'close': 61.59,
  'date': '2012-01-04',
  'feed_source': 3.0,
  'gross': 61.59,
  'high': 61.98,
  'low': 59.8,
  'nav': 61.59,
  'open': 59.97,
  'pl': 1.5500000000000043,
  'return': 0.02581612258494344,
  'volume': 676506.0},
 {'ask': 61.84,
  'bid': 61.82,
  'close': 61.86,
  'date': '2012-01-05',
  'feed_source': 3.0,
  'gross': 61.86,
  'high': 62.01,
  'low': 60.47,
  'nav': 61.86,
  'open': 61.14,
  'pl': 0.269999999999996,
  'return': 0.004383828543594675,
  'volume': 638189.0},
 {'ask': 62.61,
  'bid': 62.6,
  'close': 62.64,
  'date': '2012-01-06',
  'feed_source': 3.0,
  'gross': 62.64,
  'high': 62.94,
  'low': 61.22,
  'nav': 62.64,
  'open': 62.0,
  'pl': 0.7800000000000011,
  'return': 0.012

In [142]:
assets[0]['ASSET PRICE MODE']['value']

'Par Défaut (configuration)'